# Real footprinting data (simple case)

Rationale: After successfully evaluating the `footprint` module on artificial data, the module is to be tested on real data. To simplify evaluation of the success of this, I load a simplified version first, containing only the control # 7 (reference trace, without added ligand) and the other control traces (# 14 and 21) from MS010.

Realization:

- load all traces from HexA.csv into `trace_list`

In [1]:
import footprint
trace_list = footprint.get_data()

- fetch all traces with Ltot_conc = 0M

In [2]:
conc_0_traces = []
for t in trace_list:
    print(t.Ltot_conc,t.file_name)
    if t.Ltot_conc == 0:
        conc_0_traces.append(t)
#print(len(conc_0_traces))

3.0 103-23-16-6-33 PM.fsa
1.0 113-23-16-7-08 PM.fsa
0.3 123-23-16-7-42 PM.fsa
0.1 133-23-16-8-17 PM.fsa
0.0 143-23-16-8-51 PM.fsa
10.0 83-23-16-5-24 PM.fsa
5.0 93-23-16-5-58 PM.fsa
10.0 13-23-16-1-22 PM.fsa
5.0 23-23-16-1-56 PM.fsa
3.0 33-23-16-2-31 PM.fsa
1.0 43-23-16-3-05 PM.fsa
0.3 53-23-16-3-40 PM.fsa
0.1 63-23-16-4-15 PM.fsa
0.0 73-23-16-4-49 PM.fsa
10.0 153-23-16-9-26 PM.fsa
5.0 163-23-16-10-01 PM.fsa
3.0 173-23-16-10-35 PM.fsa
1.0 183-23-16-11-10 PM.fsa
0.3 193-23-16-11-44 PM.fsa
0.1 203-24-16-12-19 AM.fsa
0.0 213-24-16-12-54 AM.fsa


- set trace with most peaks as reference (arbitrary)

In [3]:
peaknr = [len(c.peaks) for c in conc_0_traces]

ref = [c for c in conc_0_traces if len(c.peaks) == max(peaknr)][0]

traces = [c for c in conc_0_traces if len(c.peaks) != max(peaknr)]

print(len(ref.peaks))
print(len(traces))
print(peaknr)

146
2
[139, 129, 146]


--> we have one `ref` with 146 peaks, and two other traces with less peaks...

In [4]:
footprint.cluster_peaks(ref,traces)

footprint.calculate_deviance_for_all_peaks(ref,traces)

#footprint.determine_factor_numerically(ref, traces[0])

#footprint.add_fractional_occupancies(ref,trace)

#footprint.calculate_free_ligand_concentration(ref,traces)

898.72803214185762

- check the clustered peaks

In [5]:
for i in footprint.give_all_clustered_peaks(ref,traces): 
    #print(i) ## extensive output
    #print("---")
    print(i[0].cluster,i[0].peak_height,[n.peak_height for n in i[1]]) ##compact output


1 55.0 []
2 165.0 [160.0, 107.0]
3 1159.0 [370.0, 264.0]
4 529.0 [275.0, 156.0]
5 794.0 [340.0, 218.0]
6 555.0 [404.0, 307.0]
7 7158.0 [5928.0, 4563.0]
8 193.0 [94.0, 78.0]
9 124.0 [69.0, 51.0]
10 1968.0 [837.0, 602.0]
11 330.0 [157.0, 98.0]
12 221.0 [124.0, 62.0]
13 2708.0 [1177.0, 799.0]
14 1522.0 [674.0, 448.0]
15 183.0 [81.0, 52.0]
16 887.0 [381.0, 246.0]
17 376.0 [187.0, 115.0]
18 1122.0 [514.0, 338.0]
19 905.0 [406.0, 280.0]
20 171.0 [68.0, 51.0]
21 75.0 []
22 126.0 [63.0]
23 962.0 [405.0, 278.0]
24 3048.0 [1388.0, 920.0]
25 291.0 [118.0, 79.0]
26 1644.0 [679.0, 465.0]
27 86.0 []
28 4537.0 [2014.0, 1369.0]
29 518.0 [251.0, 153.0]
30 2118.0 [917.0, 632.0]
31 65.0 []
32 558.0 [224.0, 151.0]
33 6943.0 [3020.0, 2182.0]
34 177.0 [83.0, 65.0]
35 1393.0 [601.0, 434.0]
36 481.0 [234.0, 153.0]
37 354.0 [176.0, 123.0]
38 392.0 [180.0, 130.0]
39 366.0 [204.0, 139.0]
40 240.0 []
41 251.0 [96.0, 72.0]
42 428.0 [210.0, 134.0]
43 170.0 [87.0]
44 390.0 [174.0, 121.0]
45 5790.0 [2479.0, 1743.0]
4

- find the optimal factors based on a single peak, and optimize the traces with this factor (!! NB: This is not necessarily the same peak with the current version of the `footprint` module!)

In [6]:
import imp
imp.reload(footprint)

zero = footprint.determine_factor_single_peak(ref, traces[0], 1, 1, True)
one = footprint.determine_factor_single_peak(ref, traces[1], 1,1, True)

In [7]:
print(zero)
print(one)

1.9540229885057472
2.6778115501519757


- correct the peaks with the determined factors

In [8]:
footprint.correct_peaks_with_factor(traces[0],zero)
footprint.correct_peaks_with_factor(traces[1],one)


- check the clustered peaks again

In [9]:
for i in footprint.give_all_clustered_peaks(ref,traces): 
    #print(i) ## extensive output
    #print("---")
    print(i[0].cluster,i[0].peak_height,[n.peak_height for n in i[1]], [i[0].peak_height-n.peak_height for n in i[1]]) ##compact output


1 55.0 [] []
2 165.0 [312.64367816091954, 286.5258358662614] [-147.64367816091954, -121.52583586626139]
3 1159.0 [722.9885057471265, 706.9422492401216] [436.0114942528735, 452.05775075987845]
4 529.0 [537.3563218390805, 417.7386018237082] [-8.356321839080465, 111.26139817629178]
5 794.0 [664.3678160919541, 583.7629179331307] [129.63218390804593, 210.23708206686933]
6 555.0 [789.4252873563219, 822.0881458966566] [-234.42528735632186, -267.08814589665656]
7 7158.0 [11583.448275862069, 12218.854103343465] [-4425.448275862069, -5060.854103343465]
8 193.0 [183.67816091954023, 208.8693009118541] [9.321839080459768, -15.869300911854111]
9 124.0 [134.82758620689657, 136.56838905775075] [-10.82758620689657, -12.56838905775075]
10 1968.0 [1635.5172413793105, 1612.0425531914893] [332.48275862068954, 355.95744680851067]
11 330.0 [306.7816091954023, 262.4255319148936] [23.218390804597675, 67.57446808510639]
12 221.0 [242.29885057471265, 166.0243161094225] [-21.298850574712645, 54.9756838905775]
13 

In [10]:
rmsd_zero = footprint.calculate_deviance_for_all_peaks(ref,traces[0],1,1,True)
rmsd_one = footprint.calculate_deviance_for_all_peaks(ref,traces[0],1,1,True)

In [11]:
print(rmsd_zero,rmsd_one)

0.278066859565 0.278066859565
